### 1.2- Configuration Setup:

Configuration parameters will be loaded dynamically from a JSON file. This ensures that all subject-specific and dataset-specific information is managed externally, supporting full reusability and transparency.

*You will provide the JSON configuration file details later. The notebook will be ready to load and use it as soon as it's available.*

## Step 1: Import Libraries and Set Up Modular Functions

All imports and configurations in this notebook are designed to be dynamic and modular. No subject-specific or hard-coded values are used, ensuring scientific reproducibility and easy adaptation to new datasets.

# MEG Data Preprocessing Pipeline - Subject 02

## Overview
This notebook implements a step-by-step, interpretable, and reusable preprocessing pipeline for OPM-MEG data analysis. The approach is designed for scientific research and avoids hard-coded values, making it adaptable for different datasets and subjects.

## Research Context
- **Subject**: sub-02
- **Data Type**: OPM-MEG recordings (.lvm format)

## 1- Preprocessing Steps
The pipeline will guide you through library imports, configuration loading, and file path setup. All parameters and file names will be defined dynamically for maximum flexibility.

### 1.1- Libraries:

Import all required libraries and custom functions. The notebook will use modular imports to ensure reusability and clarity.

In [14]:
# ---- LIBRARIES ----
import json
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mne
from mne.preprocessing import ICA

# Add source directory to path for custom functions
sys.path.append('../source')

# Import custom functions (modular, not hard-coded for subject)
from find_paths import get_onedrive_path
from plot_functions import (plot_channels_comparison, plot_meg_2x3_grid, plot_ica_components, plot_all_channel_power_spectra)
from config_manager import (load_and_display_config, validate_file_selection, preprocess_meg_data, remove_ica_artifacts)
from MEG_analysis_functions import apply_fastica_to_channels

### 1.2- Load configuration:

In [15]:
# ---- LOAD CONFIGURATION ----

SUBJECT_ID = "sub-02"
CONFIG_VERSION = "Version-1"

config = load_and_display_config(SUBJECT_ID, version=CONFIG_VERSION)

Configuration loaded for sub-02
SELECTABLE PARAMETERS:
DATA_TYPES: ['source_data']
SETUPS: ['MED', 'NO_MED']
CONDITIONS:
  MED: ['Move', 'Rest']
  NO_MED: ['Move', 'Rest']


### 1.3- Path Setup:

In [19]:
# ---- PATH SETUP ----

base_data_path = get_onedrive_path('data')
print("Base data path found successfully")
print(f"Base path: {base_data_path}")

# ---- FILE SELECTION ----

print("\n" + "="*60)
print("Based on the available options above, define your selection:")

# USER DEFINES THESE PARAMETERS:
DATA_TYPE = "source_data" 
SETUP = 'NO_MED'  # "MED" or "NO_MED"
CONDITION = config["conditions"][SETUP][0]  # "Move" or "Rest"
TIME_POINT = config["time_points"][0]  # select a number from -1 to 7 (0 excluded)

print(f"\nSelected: {SETUP} -> {CONDITION} -> {TIME_POINT}")

# Validate selection and construct filename using config
filename, path_info = validate_file_selection(config, DATA_TYPE, SETUP, CONDITION, TIME_POINT)


# Construct complete file path using ALL config parameters
file_path = os.path.join(
    base_data_path,
    config["data_path"],
    filename
)

print(f"Selected file: {filename}")
print(f"Full file path: {file_path}")
print(f"File exists: {os.path.exists(file_path)}")

project folder found: /Users/federicobonato/OneDrive - Charité - Universitätsmedizin Berlin/TRR295 via Teams - LID_MEG - LID_MEG
Base data path found successfully
Base path: /Users/federicobonato/OneDrive - Charité - Universitätsmedizin Berlin/TRR295 via Teams - LID_MEG - LID_MEG/data

Based on the available options above, define your selection:

Selected: NO_MED -> Move -> -1
Selected file: sub02-NO_MED-Move_-1.lvm
Full file path: /Users/federicobonato/OneDrive - Charité - Universitätsmedizin Berlin/TRR295 via Teams - LID_MEG - LID_MEG/data/source_data/sub-02/plfp65/sub02-NO_MED-Move_-1.lvm
File exists: True


In [21]:
if os.path.exists(file_path):
    print("\nPreview of file content:")
    with open(file_path, "r") as f:
        for i in range(30):  # Print first 10 lines for a quick look
            line = f.readline()
            if not line:
                break
            print(line.strip())
else:
    print("\nSelected file does not exist!")
    print(f"Checked path: {file_path}")


Preview of file content:
LabVIEW Measurement
Writer_Version	2
Reader_Version	2
Separator	Tab
Decimal_Separator	.
Multi_Headings	No
X_Columns	One
Time_Pref	Absolute
Operator	Neuro1
Date	2024/11/13
Time	12:44:10.1353811994194984436
***End_of_Header***

Channels	224
Samples	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16	16
Date	2024/11/13	2024/11/13	2024/11/13	2024/11/13	2024/1